<a href="https://colab.research.google.com/github/and-is/learning-pytorch/blob/main/hyperparamTuning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We're trying to find best values for the following parameters:
- Number of hidden layers
- Neurons per layer
- No of epochs
- Optimizer
- Learning rate
- Batch size
- Dropout rate
- Weight decay

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
df = pd.read_csv('fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [ ]:
train_dataset = CustomDataset(X_train, y_train)

In [ ]:
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []

    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))

      # has a glitch, so can't use different no of neurons in different hidden layers
      input_dim = neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)


In [ ]:
# Now we'll build an objective function for our Hyperparameter tuning. We'll first perform on only no of layers and no of neurons.
def objective(trial):

  # next hyperparameter values extract from search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

  # dropout needs to be passed via function call now to neural network definition instead of hardcoding there.
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

  # batch size is set during data loader class which needs to be managed by placing dataloader code inside objective function.
  batch_size = trial.suggest_categorical("batch_size", [16,32,64,128])

  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 784
  output_dim = 10
  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # loss function
  criterion = nn.CrossEntropyLoss()

  # select optimizer
  if optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


  # training loop
  for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
      outputs = model(batch_features)
      loss = criterion(outputs, batch_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  # evaluation
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for batch_features, batch_labels in test_loader:
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
      outputs = model(batch_features)
      _, predicted = torch.max(outputs, 1)
      total += batch_labels.shape[0]
      correct += (predicted == batch_labels).sum().item()
    accuracy = correct/total

  return accuracy

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-02-03 15:35:13,268] A new study created in memory with name: no-name-64ad5614-516b-4656-9cac-fc8863730c71


In [ ]:
study.optimize(objective, n_trials=10)

[I 2025-02-03 15:57:27,212] Trial 0 finished with value: 0.8248333333333333 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 120, 'epochs': 50, 'learning_rate': 0.009429382287269179, 'dropout_rate': 0.1, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.00036994692324769906}. Best is trial 0 with value: 0.8248333333333333.
[I 2025-02-03 15:58:17,111] Trial 1 finished with value: 0.8806666666666667 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 72, 'epochs': 10, 'learning_rate': 0.00562698884608015, 'dropout_rate': 0.1, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.00010167882627540535}. Best is trial 1 with value: 0.8806666666666667.
[I 2025-02-03 16:05:58,603] Trial 2 finished with value: 0.8245 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 104, 'epochs': 20, 'learning_rate': 0.012438112824993844, 'dropout_rate': 0.4, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 5.335811959496441e-05}. Best is trial 1 with value

In [ ]:
study.best_value

In [ ]:
study.best_params

For future references:
\
Check Optuna X MLFlow for better results.